# Ag-Analytics® - SSURGO Soils API 

The Soil Survey Geographic Database (SSURGO) Soils API provides soil type and soil type
attribute averages (e.g., NCCPI) by field for a shape entered. This API uses POST and GET request with a subscription key. Soil Type data is derived from
NRCS USDA; average soil type attribute data are available for processing for the continental
USA. It uses data provided by the USDA NRCS, which can be downloaded from
http://sdmdataaccess.nrcs.usda.gov. Those soil type attributes contain both numerical data type
and categorical (ordinal) data type, which can be found from the links in the reference. 

### Post Request Parameters

1). __AOI__ (GeoJSON String): Area of interest as Geojson string containing boundary coordinates.
    __file__(GeoTiff or Shapfile in Zip): Alternative AOI format when GeoJSON is not available. A GeoTiff(.TIFF,.Tif, .etc) or a Shapefile in Zip (must include shp, shx, dbf, but others files such as .prj, .xml, .cpg are recommended for better processing)
    
2).__Projection__(String):Output projection of result GeoTIFF.Provided as EPSG code, ex: "EPSG:4326"

3).__Resolution__(Float):Output resolution of result GeoTIFF.

4).__Soil_Parameter__(String): The soil metric to return, see the [NRCS variables PDF](https://apimgmtstkfuxw9ek5vx5grk.blob.core.windows.net/content/MediaLibrary/SSURGO/nrcs142p2_052165.pdf) for full list with descriptions

## Library

In [1]:
import requests
import json
import os
from collections import defaultdict
import pandas as pd
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


## POST Request API Fuction

In [25]:
def ssurgo_service(values, files, headers):
    try:
        url = 'https://ag-analytics.azure-api.net/ssurgo-v2'
        
        if files == None:
            response = requests.post(url, data=values,headers = headers).json()
        else:
            response = requests.post(url, files = files, data = values,headers = headers).json()
            
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

## 1. GeoJSON as AOI

In [29]:
#Parameters to call SSURGO API
values = {'AOI': '{"geometryType": "esriGeometryPolygon", "features": [{"geometry": {"rings": [[[-85.179, 42.74], [-85.17858886748223, 42.74188232450973], [-85.17858886748223, 42.742675781062474], [-85.1782836915391, 42.742675781062474], [-85.1782226563505, 42.74230956993074], [-85.17529296909521, 42.74230956993074], [-85.17529296909521, 42.74353027370324], [-85.17529296909521, 42.74371337926908], [-85.17492675796348, 42.74389648393566], [-85.17437744126585, 42.744079589501496], [-85.17340087914721, 42.744079589501496], [-85.17327880876996, 42.74749755826576], [-85.17401123013411, 42.74749755826576], [-85.17401123013411, 42.74847412128372], [-85.17529296909521, 42.74847412128372], [-85.17590332008211, 42.74829101571788], [-85.17749023408697, 42.74792480458609], [-85.17761230446422, 42.7470703128447], [-85.1782836915391, 42.746704101712965], [-85.18072509728535, 42.746704101712965], [-85.179, 42.74]]], "spatialReference": {"wkid": 4326}}}]}', 
'Soil_Parameter': 'nccpi2all',
'Projection': 'EPSG:4326', 
'Resolution': 0.00001,
'Product':'GeoJSON'}
file = None

# Basic Header Pattern.
# headers=None
# Header for using a subscription key.
# headers={'Content-Type':'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

#### Call SSURGO API

In [31]:
SSURGO_response_GeoJSON = ssurgo_service(values, file, headers)

{'attributes': {'GeoJSON': {'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}, 'type': 'name'}, 'features': [{'geometry': {'coordinates': [[[-85.17364994099194, 42.74739381697111], [-85.17377913220116, 42.74742650822685], [-85.17387346748649, 42.747493420485], [-85.17387584382337, 42.74749755861743], [-85.17348729857176, 42.74749755874103], [-85.1734504165035, 42.747386498288336], [-85.17364994099194, 42.74739381697111]]], 'type': 'Polygon'}, 'properties': {'Soil_Type': 'Udorthents and Udipsamments, 0 to 6 percent slopes', 'area': 311.26838986575603, 'mukey': 186064.0, 'nccpi2all': 0.0, 'w_nccpi2al': 0.0}, 'type': 'Feature'}, {'geometry': {'coordinates': [[[-85.179, 42.74], [-85.17910007727738, 42.740388962302006], [-85.17892976287567, 42.74032158106214], [-85.179, 42.74]]], 'type': 'Polygon'}, 'properties': {'Soil_Type': 'Glendora loamy fine sand', 'area': 270.62486958503723, 'mukey': 186065.0, 'nccpi2all': 0.123, 'w_nccpi2al': 33.28685895895958}, 'type': 'Feature'}, {'g

## 2. Raster (.tif) as AOI

In [43]:
#Path to raster in your local machine
# rasterpath = r"raster path on your local machine"
rasterpath = r"C:\Users\Xuewei\Documents\Xueweilocal\test_raster\test_raster.tif"
files = [('file',(open(rasterpath, 'rb')))]
values = {'Soil_Parameter': 'nccpi2all',
'Projection': 'EPSG:4326', 
'Resolution': 0.0001,
'Product':'raster'}
headers={'Ocp-Apim-Subscription-Key': "XXXXXXXXXXXXXXXXXXXXXXXXXX"}

#### Call SSURGO API

In [44]:
SSURGO_response_raster = ssurgo_service(values, files, headers)

{'attributes': {'CellSize': [0.0001, -0.0001], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-90.60760921213281, 39.10684154247035, -90.59080921213281, 39.12204154247035', 'Legend': [{'Acres': 93006.0046249134, 'Area': '8.8 %', 'Count': 906, 'CountAllPixels': 10296, 'Soil_Type': 'Water', 'Soil_Value': 0.0, 'Value': 175536.0, 'color': '#5ec68d'}, {'Acres': 65024.39249074459, 'Area': '6.45 %', 'Count': 664, 'CountAllPixels': 10296, 'Soil_Type': 'Water', 'Soil_Value': 0.0, 'Value': 211794.0, 'color': '#304a33'}, {'Acres': 63124.534493334744, 'Area': '6.01 %', 'Count': 619, 'CountAllPixels': 10296, 'Soil_Type': 'Water', 'Soil_Value': 0.0, 'Value': 728317.0, 'color': '#0a7fad'}, {'Acres': 274435.64372268086, 'Area': '26.86 %', 'Count': 2765, 'CountAllPixels': 10296, 'Soil_Type': 'Tice silty clay loa

## Visualize the response

#### 1. GeoJSON's product

In [37]:
# import required library

import folium
import mplleaflet

In [38]:
#Display result GeoJSON on map
result_geojson = SSURGO_response_GeoJSON['attributes']['GeoJSON']
zoom_point=result_geojson['features'][0]['geometry']['coordinates'][0][0]
zoom_point
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in result_geojson['features']:
    folium.GeoJson(shape).add_to(m)
m

In [41]:
result_geojson = SSURGO_response_GeoJSON['attributes']['GeoJSON']
zoom_point=result_geojson['features'][0]['geometry']['coordinates'][0][0]
zoom_point
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in result_geojson['features']:
    m.add_child(folium.GeoJson(data=shape).add_child(folium.Popup(str(shape["properties"]))))
m


#### 2. Raster's product

In [34]:
# pngb64
attributes =SSURGO_response_raster['attributes']
pngb64 = attributes['pngb64']

In [35]:
#Image Legend
df=defaultdict(list)

Legend = attributes['Legend']
    
for val in Legend:
    df['Acres'].append(float(val['Acres']))
    df['Area'].append(val['Area'])
    df['Value'].append(val['Value'])
    df['Soil_Type'].append(val['Soil_Type'])
    df['color'].append(val['color'])

    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['color'])

,Acres,Area,Value,Soil_Type,color
0,93006,8.8 %,175536,Water,#05940c
1,65024.4,6.45 %,211794,Water,#665298
2,63124.5,6.01 %,728317,Water,#5533c1
3,274436,26.86 %,728434,"Tice silty clay loam, 0 to 2 percent slopes, occasionally flooded",#be0ea9
4,208908,23.37 %,728461,"Beaucoup silty clay loam, cool mesic, 0 to 2 percent slopes, occasionally flooded",#5a5176
5,28523.8,3.9 %,728463,"Quiver silty clay loam, 0 to 2 percent slopes, frequently flooded, long duration",#c8b5f4
6,89701.6,10.74 %,728483,"McFain silty clay, 0 to 2 percent slopes, occasionally flooded",#10f28a
7,105496,11.26 %,2.54204e+06,"Beaucoup silty clay loam, undrained, 0 to 2 percent slopes, frequently flooded, long duration",#1a6557
8,15505.6,2.0 %,2.54228e+06,"Quiver silty clay loam, 0 to 2 percent slopes, frequently flooded, long duration",#018111
9,3643.72,0.61 %,2.54229e+06,"Beaucoup silty clay loam, cool mesic, 0 to 2 percent slopes, occasionally flooded",#aca535


In [36]:
# Visualize CDL
Image(url = pngb64, width = 400, height = 400)

## GET Request

In [46]:
# Specify local path where file will be downloaded.
# local_path = r"C:\<Path to download directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"
local_path = r'C:'
# Create GET request payload.
values = {'filename': SSURGO_response_raster['filename']}
print(values)

{'filename': 'result_ssurgo_raster_proj_4326_res_0.0001_20200717171310522516.tif'}


In [50]:
# Get SSURGO file 
def SSURGO_get(values, local_path,headers):
    try:
        url = 'https://ag-analytics.azure-api.net/ssurgo-v2'
    
        download_path = os.path.join(local_path, values['filename'])
        print(values)
     
        response = requests.get(url, params=values,headers=headers)
        open(download_path, 'wb').write(response.content)
        
        print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [51]:
SSURGO_get(values, local_path,headers)

{'filename': 'result_ssurgo_raster_proj_4326_res_0.0001_20200717171310522516.tif'}
https://ag-analytics.azure-api.net/ssurgo-v2?filename=result_ssurgo_raster_proj_4326_res_0.0001_20200717171310522516.tif


<Response [200]>